In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [57]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-0.028872,-0.130254,-3.184231e+07,-3.115075e+07,-7184.203318,...,-170775.886453,-1059.042606,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.016772,-0.034656,-1.968379e+06,-1.924910e+06,-451.342112,...,-11502.243880,-168.146428,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.015297,-0.094878,-6.059927e+07,-5.928166e+07,-13774.014902,...,-324959.759255,-3804.051175,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.014625,NaN,-2.290124e+06,-2.239814e+06,-510.210093,...,-12199.928822,-901.413470,-0.189720,-1.009642,-253.366805,-0.226133,-3.169102,-3.169102,-92.619013,0.016390
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.006993,-0.050957,-3.997865e+07,-3.910761e+07,-9330.459280,...,-214438.078273,-4116.308632,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005


In [19]:
GICS_df = df[df['GICS Sub-Industry'] == 'Brewers']
GICS_df

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
514,2019,CHINA RESOURCES BEER (HLDGS),HONG KONG,Brewers,Manufacture of beverages,-0.122360,-1.772142,-5.919396e+08,-9.959571e+07,-24068.773953,...,-4.906714e+08,-16055.630122,-202.440777,-7.429111,-11784.460202,-241.295146,2951.887926,2951.887926,-4146.557853,0.000931
517,2019,COMPANIA CERVECERIAS UNIDAS,CHILE,Brewers,Manufacture of beverages,-0.050090,-0.422750,-1.214176e+08,-2.630367e+07,-4997.667428,...,-9.467504e+07,-65916.007408,-101.109045,-168.330751,-2127.077366,-120.514860,1091.813734,-52690.868311,-850.968779,0.084882
518,2019,BUDWEISER BREWING CO,CAYMAN ISLANDS,Brewers,Manufacture of beverages,-0.049434,-0.265065,-3.254224e+08,-7.637925e+07,-19237.321007,...,-2.477599e+08,-21847.771988,-136.359349,-10.109197,-9442.658019,-162.530738,1960.864931,1960.864931,-3199.497695,0.017490
519,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-0.048069,-0.339027,-5.085402e+08,-1.099335e+08,-29928.965649,...,-3.967482e+08,-77571.792798,-1080.784815,-117.835584,-13512.921279,-1288.219368,16032.014249,16032.014249,-4470.382055,0.059202
525,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-0.014656,-0.049706,-7.692113e+08,-5.738226e+08,-148508.142003,...,-1.854630e+08,-222519.248882,-3800.282607,-117.403486,-69595.453393,-4529.669170,56040.408797,56040.408797,-23485.242748,0.080199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13371,2010,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-0.011646,-0.074848,-1.259374e+08,-8.266123e+07,-46875.750497,...,-4.167691e+07,-390154.909705,-3135.807631,-110.758712,-19572.986856,-3737.661806,45414.814856,45285.736294,-3683.062568,0.020611
13372,2010,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.011321,-0.087939,-2.450821e+08,-1.936780e+08,-67850.075481,...,-4.809901e+07,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366,0.008737
13374,2010,KIRIN HOLDINGS CO LTD,JAPAN,Brewers,Manufacture of beverages,-0.007504,-0.107791,-2.015159e+08,-1.831167e+08,-44382.550185,...,-1.532630e+07,-37042.309634,-551.651117,-38.777747,-21482.424506,-657.529272,7924.919052,7924.919052,-7563.043320,0.006961
13378,2010,ASAHI GROUP HOLDINGS LTD,JAPAN,Brewers,Manufacture of beverages,-0.004985,-0.077865,-9.154859e+07,-8.336182e+07,-19112.482311,...,-6.784045e+06,-25334.276719,-72.372578,-26.521191,-9431.094408,-86.263015,913.746217,913.746217,-3419.723102,0.011001


In [20]:
# Group the data by "Company Name" and count unique years for each group
company_years_count = GICS_df.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
filtered_df = GICS_df[GICS_df["Company Name"].isin(filtered_companies)]

In [21]:
new_df = filtered_df[["Company Name","Environmental Intensity (Sales)",
                      "Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
new_df = new_df.reset_index()
new_df = new_df.drop("index", axis=1)
new_df

,Company Name,Environmental Intensity (Sales),Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity
0,MOLSON COORS BEVERAGE CO,-0.048069,-1.099335e+08,-29928.965649,-1.448611e+06,-3.288817e+05,-5564.578068,-77571.792798,-3.967482e+08,32064.028497
1,ANHEUSER-BUSCH INBEV,-0.014656,-5.738226e+08,-148508.142003,-7.892568e+06,-1.746848e+06,-27332.682146,-222519.248882,-1.854630e+08,112080.817593
2,CARLSBERG A/S,-0.011643,-7.527218e+07,-24932.542858,-9.632243e+05,-2.175576e+05,-4576.639315,-366663.271634,-3.834863e+07,40738.331463
3,HEINEKEN HOLDING NV,-0.008577,-1.593026e+08,-43440.457121,-2.154098e+06,-4.856821e+05,-7979.660298,-198189.995171,-6.833007e+07,41453.221269
4,HEINEKEN NV,-0.006680,-1.594139e+08,-43465.922058,-2.155579e+06,-4.860293e+05,-7984.294936,-198189.995171,-1.724705e+07,41453.221269
...,...,...,...,...,...,...,...,...,...,...
65,HEINEKEN HOLDING NV,-0.014876,-1.936780e+08,-67850.075481,-2.547754e+06,-5.790435e+05,-10648.573449,-165536.025577,-1.250402e+08,65724.388330
66,CARLSBERG A/S,-0.011646,-8.266123e+07,-46875.750497,-1.020688e+06,-2.253744e+05,-6986.860242,-390154.909705,-4.167691e+07,90829.629712
67,HEINEKEN NV,-0.011321,-1.936780e+08,-67850.075481,-2.547754e+06,-5.790435e+05,-10648.573449,-165536.025577,-4.809901e+07,65724.388330
68,KIRIN HOLDINGS CO LTD,-0.007504,-1.831167e+08,-44382.550185,-2.433998e+06,-5.651874e+05,-8121.539743,-37042.309634,-1.532630e+07,15849.838105


In [22]:
with_rev = filtered_df.copy()
with_rev

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
519,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-0.048069,-0.339027,-5.085402e+08,-1.099335e+08,-29928.965649,...,-3.967482e+08,-77571.792798,-1080.784815,-117.835584,-13512.921279,-1288.219368,16032.014249,16032.014249,-4470.382055,0.059202
525,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-0.014656,-0.049706,-7.692113e+08,-5.738226e+08,-148508.142003,...,-1.854630e+08,-222519.248882,-3800.282607,-117.403486,-69595.453393,-4529.669170,56040.408797,56040.408797,-23485.242748,0.080199
527,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-0.011643,-0.077417,-1.151570e+08,-7.527218e+07,-24932.542858,...,-3.834863e+07,-366663.271634,-1437.329515,-104.089813,-10614.776651,-1713.195534,20369.165732,20247.859126,-3000.167719,0.029599
532,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.008577,-0.056102,-2.304806e+08,-1.593026e+08,-43440.457121,...,-6.833007e+07,-198189.995171,-1416.040953,-60.818961,-19958.750361,-1687.821069,20726.610635,20726.610635,-6546.048034,0.093336
535,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.006680,-0.041057,-1.795108e+08,-1.594139e+08,-43465.922058,...,-1.724705e+07,-198189.995171,-1416.040953,-60.818961,-19971.482830,-1687.821069,20726.610635,20726.610635,-6550.682671,0.101854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13368,2010,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.014876,-0.115547,-3.220233e+08,-1.936780e+08,-67850.075481,...,-1.250402e+08,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366,0.151424
13371,2010,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-0.011646,-0.074848,-1.259374e+08,-8.266123e+07,-46875.750497,...,-4.167691e+07,-390154.909705,-3135.807631,-110.758712,-19572.986856,-3737.661806,45414.814856,45285.736294,-3683.062568,0.020611
13372,2010,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.011321,-0.087939,-2.450821e+08,-1.936780e+08,-67850.075481,...,-4.809901e+07,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366,0.008737
13374,2010,KIRIN HOLDINGS CO LTD,JAPAN,Brewers,Manufacture of beverages,-0.007504,-0.107791,-2.015159e+08,-1.831167e+08,-44382.550185,...,-1.532630e+07,-37042.309634,-551.651117,-38.777747,-21482.424506,-657.529272,7924.919052,7924.919052,-7563.043320,0.006961


In [24]:
with_rev["Revenue"] = with_rev["Total Environmental Cost"]/with_rev["Environmental Intensity (Sales)"]
with_rev

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue
519,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-0.048069,-0.339027,-5.085402e+08,-1.099335e+08,-29928.965649,...,-77571.792798,-1080.784815,-117.835584,-13512.921279,-1288.219368,16032.014249,16032.014249,-4470.382055,0.059202,1.057940e+10
525,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-0.014656,-0.049706,-7.692113e+08,-5.738226e+08,-148508.142003,...,-222519.248882,-3800.282607,-117.403486,-69595.453393,-4529.669170,56040.408797,56040.408797,-23485.242748,0.080199,5.248464e+10
527,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-0.011643,-0.077417,-1.151570e+08,-7.527218e+07,-24932.542858,...,-366663.271634,-1437.329515,-104.089813,-10614.776651,-1713.195534,20369.165732,20247.859126,-3000.167719,0.029599,9.890572e+09
532,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.008577,-0.056102,-2.304806e+08,-1.593026e+08,-43440.457121,...,-198189.995171,-1416.040953,-60.818961,-19958.750361,-1687.821069,20726.610635,20726.610635,-6546.048034,0.093336,2.687117e+10
535,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.006680,-0.041057,-1.795108e+08,-1.594139e+08,-43465.922058,...,-198189.995171,-1416.040953,-60.818961,-19971.482830,-1687.821069,20726.610635,20726.610635,-6550.682671,0.101854,2.687117e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13368,2010,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.014876,-0.115547,-3.220233e+08,-1.936780e+08,-67850.075481,...,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366,0.151424,2.164753e+10
13371,2010,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-0.011646,-0.074848,-1.259374e+08,-8.266123e+07,-46875.750497,...,-390154.909705,-3135.807631,-110.758712,-19572.986856,-3737.661806,45414.814856,45285.736294,-3683.062568,0.020611,1.081400e+10
13372,2010,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-0.011321,-0.087939,-2.450821e+08,-1.936780e+08,-67850.075481,...,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366,0.008737,2.164753e+10
13374,2010,KIRIN HOLDINGS CO LTD,JAPAN,Brewers,Manufacture of beverages,-0.007504,-0.107791,-2.015159e+08,-1.831167e+08,-44382.550185,...,-37042.309634,-551.651117,-38.777747,-21482.424506,-657.529272,7924.919052,7924.919052,-7563.043320,0.006961,2.685426e+10


In [25]:
with_rev_un = with_rev[["Company Name","Revenue",
                      "Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
with_rev_un = with_rev_un.reset_index()
with_rev_un = with_rev_un.drop("index", axis=1)
with_rev_un

,Company Name,Revenue,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity
0,MOLSON COORS BEVERAGE CO,1.057940e+10,-1.099335e+08,-29928.965649,-1.448611e+06,-3.288817e+05,-5564.578068,-77571.792798,-3.967482e+08,32064.028497
1,ANHEUSER-BUSCH INBEV,5.248464e+10,-5.738226e+08,-148508.142003,-7.892568e+06,-1.746848e+06,-27332.682146,-222519.248882,-1.854630e+08,112080.817593
2,CARLSBERG A/S,9.890572e+09,-7.527218e+07,-24932.542858,-9.632243e+05,-2.175576e+05,-4576.639315,-366663.271634,-3.834863e+07,40738.331463
3,HEINEKEN HOLDING NV,2.687117e+10,-1.593026e+08,-43440.457121,-2.154098e+06,-4.856821e+05,-7979.660298,-198189.995171,-6.833007e+07,41453.221269
4,HEINEKEN NV,2.687117e+10,-1.594139e+08,-43465.922058,-2.155579e+06,-4.860293e+05,-7984.294936,-198189.995171,-1.724705e+07,41453.221269
...,...,...,...,...,...,...,...,...,...,...
65,HEINEKEN HOLDING NV,2.164753e+10,-1.936780e+08,-67850.075481,-2.547754e+06,-5.790435e+05,-10648.573449,-165536.025577,-1.250402e+08,65724.388330
66,CARLSBERG A/S,1.081400e+10,-8.266123e+07,-46875.750497,-1.020688e+06,-2.253744e+05,-6986.860242,-390154.909705,-4.167691e+07,90829.629712
67,HEINEKEN NV,2.164753e+10,-1.936780e+08,-67850.075481,-2.547754e+06,-5.790435e+05,-10648.573449,-165536.025577,-4.809901e+07,65724.388330
68,KIRIN HOLDINGS CO LTD,2.685426e+10,-1.831167e+08,-44382.550185,-2.433998e+06,-5.651874e+05,-8121.539743,-37042.309634,-1.532630e+07,15849.838105


In [26]:
with_rev_sdg = with_rev[["Company Name","Revenue",
                         "SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]]
with_rev_sdg = with_rev_sdg.reset_index()
with_rev_sdg = with_rev_sdg.drop("index", axis=1)
with_rev_sdg

,Company Name,Revenue,SDG 1.5,SDG 2.1,SDG 2.2,SDG 2.3,SDG 2.4,SDG 3.3,SDG 3.4,SDG 3.9,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5
0,MOLSON COORS BEVERAGE CO,1.057940e+10,-4.729036e+07,-2.991912e+07,-2.989485e+07,-3.621528e+05,-3.621528e+05,-4.856362e+06,9.693671e+05,-10362.284654,-3.967482e+08,-77571.792798,-1080.784815,-117.835584,-13512.921279,-1288.219368,16032.014249,16032.014249,-4470.382055
1,ANHEUSER-BUSCH INBEV,5.248464e+10,-2.510232e+08,-1.546841e+08,-1.544417e+08,-1.973137e+06,-1.973337e+06,-2.502910e+07,5.599867e+06,-11693.577522,-1.854630e+08,-222519.248882,-3800.282607,-117.403486,-69595.453393,-4529.669170,56040.408797,56040.408797,-23485.242748
2,CARLSBERG A/S,9.890572e+09,-3.131879e+07,-2.185492e+07,-2.183969e+07,-2.407823e+05,-2.410946e+05,-3.576818e+06,2.642967e+06,-36341.298323,-3.834863e+07,-366663.271634,-1437.329515,-104.089813,-10614.776651,-1713.195534,20369.165732,20247.859126,-3000.167719
3,HEINEKEN HOLDING NV,2.687117e+10,-6.983259e+07,-4.401123e+07,-4.396553e+07,-5.385245e+05,-5.385245e+05,-7.139792e+06,4.074973e+06,-12898.398460,-6.833007e+07,-198189.995171,-1416.040953,-60.818961,-19958.750361,-1687.821069,20726.610635,20726.610635,-6546.048034
4,HEINEKEN NV,2.687117e+10,-6.988243e+07,-4.404021e+07,-4.399450e+07,-5.388947e+05,-5.388947e+05,-7.144462e+06,4.074973e+06,-12898.398460,-1.724705e+07,-198189.995171,-1416.040953,-60.818961,-19971.482830,-1687.821069,20726.610635,20726.610635,-6550.682671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HEINEKEN HOLDING NV,2.164753e+10,-8.341504e+07,-5.691220e+07,-5.685826e+07,-6.369386e+05,-6.369386e+05,-9.296969e+06,1.092836e+07,-10773.246225,-1.250402e+08,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366
66,CARLSBERG A/S,1.081400e+10,-3.268369e+07,-2.964878e+07,-2.962240e+07,-2.551467e+05,-2.554790e+05,-4.940469e+06,1.351385e+07,-38669.637955,-4.167691e+07,-390154.909705,-3135.807631,-110.758712,-19572.986856,-3737.661806,45414.814856,45285.736294,-3683.062568
67,HEINEKEN NV,2.164753e+10,-8.341504e+07,-5.691220e+07,-5.685826e+07,-6.369386e+05,-6.369386e+05,-9.296969e+06,1.092836e+07,-10773.246225,-4.809901e+07,-165536.025577,-2234.946175,-50.798372,-31212.674683,-2663.898408,32862.194165,32862.194165,-8385.894366
68,KIRIN HOLDINGS CO LTD,2.685426e+10,-8.117517e+07,-4.846926e+07,-4.844198e+07,-6.084994e+05,-6.084994e+05,-7.829233e+06,9.990386e+05,-4477.143978,-1.532630e+07,-37042.309634,-551.651117,-38.777747,-21482.424506,-657.529272,7924.919052,7924.919052,-7563.043320


In [27]:
X = with_rev_un[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
y = np.log(with_rev_un[["Revenue"]])
X = sm.add_constant(X)

In [28]:
X_sdg = with_rev_sdg[["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]]
y_sdg = np.log(with_rev_sdg[["Revenue"]])
X_sdg = sm.add_constant(X_sdg)

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the Lasso regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

# Get the coefficients (weights) of the independent variables
coefficients = lasso_model.coef_

# Print the coefficients
print("Lasso Coefficients:")
for i in range(len(coefficients)):
    print(f"X{i+1}: {coefficients[i]:.4f}")

Lasso Coefficients:
X1: 0.0000
X2: -0.0000
X3: 0.0000
X4: -0.0000
X5: 0.0000
X6: 0.0000
X7: -0.0000
X8: -0.0000
X9: 0.0000


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.986e+00, tolerance: 1.149e-02
  model = cd_fast.enet_coordinate_descent(


In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sdg, y_sdg, test_size=0.2, random_state=42)

# Create and fit the Lasso regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

# Get the coefficients (weights) of the independent variables
coefficients = lasso_model.coef_

# Print the coefficients
print("Lasso Coefficients:")
for i in range(len(coefficients)):
    print(f"X{i+1}: {coefficients[i]:.4f}")

Lasso Coefficients:
X1: 0.0000
X2: -0.0000
X3: -0.0000
X4: 0.0000
X5: -0.0000
X6: -0.0000
X7: 0.0000
X8: -0.0000
X9: -0.0000
X10: -0.0000
X11: 0.0000
X12: 0.0000
X13: 0.0054
X14: 0.0000
X15: 0.0000
X16: 0.0001
X17: 0.0000
X18: -0.0000


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.069e+00, tolerance: 1.149e-02
  model = cd_fast.enet_coordinate_descent(


In [37]:
# Fit the random-effects panel regression model
model = sm.OLS(y, X).fit()

# Get the regression results
results = model.summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     90.04
Date:                Mon, 02 Oct 2023   Prob (F-statistic):           7.37e-40
Time:                        12:27:29   Log-Likelihood:                -54.941
No. Observations:                 100   AIC:                             127.9
Df Residuals:                      91   BIC:                             151.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

In [38]:
# Fit the random-effects panel regression model
model = sm.OLS(y_sdg, X_sdg).fit()

# Get the regression results
results = model.summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     73.59
Date:                Mon, 02 Oct 2023   Prob (F-statistic):           1.21e-38
Time:                        12:27:54   Log-Likelihood:                -53.002
No. Observations:                 100   AIC:                             128.0
Df Residuals:                      89   BIC:                             156.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.6093      0.095    217.579      0.0

## FIXED EFFECTS

In [109]:
import pandas as pd
from linearmodels.panel import PanelOLS

fixedEffects_df = df.copy()
fixedEffects_df['Year'] = pd.to_datetime(fixedEffects_df['Year'], format='%Y')

# Set 'Year' as the time variable and 'Company Name' as the entity variable
fixedEffects_df.set_index(['Industry (Exiobase)', 'Year'], inplace=True)
#fixedEffects_df["Revenue"] = fixedEffects_df["Total Environmental Cost"]/fixedEffects_df["Environmental Intensity (Sales)"]
fixedEffects_df.head()

Company Name  \
Industry (Exiobase)                                Year                             
Activities auxiliary to financial intermediatio... 2019-01-01            SAGA PLC   
                                                   2019-01-01  BURSA MALAYSIA BHD   
                                                   2019-01-01  INTERTEK GROUP PLC   
                                                   2019-01-01         JSE LIMITED   
                                                   2019-01-01   BUREAU VERITAS SA   

                                                                                                         Country  \
Industry (Exiobase)                                Year                                                            
Activities auxiliary to financial intermediatio... 2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                           MALAYSIA   
                                                   2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                       SOUTH AFRICA   
                                                   2019-01-01                                             FRANCE   

                                                                            GICS Sub-Industry  \
Industry (Exiobase)                                Year                                         
Activities auxiliary to financial intermediatio... 2019-01-01            Multi-line Insurance   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   

                                                               Environmental Intensity (Sales)  \
Industry (Exiobase)                                Year                                          
Activities auxiliary to financial intermediatio... 2019-01-01                        -0.028872   
                                                   2019-01-01                        -0.016772   
                                                   2019-01-01                        -0.015297   
                                                   2019-01-01                        -0.014625   
                                                   2019-01-01                        -0.006993   

                                                               Environmental Intensity (Op Inc)  \
Industry (Exiobase)                                Year                                           
Activities auxiliary to financial intermediatio... 2019-01-01                         -0.130254   
                                                   2019-01-01                         -0.034656   
                                                   2019-01-01                         -0.094878   
                                                   2019-01-01                               NaN   
                                                   2019-01-01                         -0.050957   

                                                               Total Environmental Cost  \
Industry (Exiobase)                                Year                                   
Activities auxiliary to financial intermediatio... 2019-01-01             -3.184231e+07   
                                                   2019-01-01             -1.968379e+06   
                                                   2019-01-01             -6.059927e+07   
                                                   2019-01-01             -2.290124e+06   
                                                   20

In [108]:

# Define the dependent variable (Y) and independent variables (X1-X15)
#y = np.log(fixedEffects_df['Revenue'])
fixedEffects_df['Environmental Intensity (Sales)'] = fixedEffects_df['Environmental Intensity (Sales)']* 100
y = fixedEffects_df['Environmental Intensity (Sales)']
X = fixedEffects_df[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]

fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Environmental Intensity (Sales),R-squared:,0.1826
Estimator:,PanelOLS,R-squared (Between):,0.2766
No. Observations:,14035,R-squared (Within):,0.1823
Date:,"Mon, Oct 02 2023",R-squared (Overall):,0.2603
Time:,15:34:00,Log-likelihood,-6.088e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,389.99
Entities:,56,P-value,0.0000
Avg Obs:,250.62,Distribution:,"F(8,13962)"
Min Obs:,1.0000,,
Max Obs:,1463.0,F-statistic (robust):,389.99
